In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import os
from datetime import datetime

# ================================
# مسیرها
# ================================
INPUT_FILE  = r"C:\BI\KZCCPP\G11\Lube Oil System\lube_oil_system_data_g11.xlsx"
OUTPUT_FILE = r"C:\BI\KZCCPP\G11\Lube Oil System\regression_output_for_assetid_9286.xlsx"
LOG_FILE    = r"C:\BI\KZCCPP\G11\Lube Oil System\regression_final_with_datetime_log.txt"

def log(msg):
    print(msg)
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {msg}\n")

log("شروع پیش‌بینی AssetID_9286 — با RecordDate + RecordTime + id + حفظ ترتیب")

# ================================
# 1. خواندن فایل اصلی (با حفظ همه ستون‌ها و ترتیب)
# ================================
if not os.path.exists(INPUT_FILE):
    log(f"خطا: فایل ورودی پیدا نشد → {INPUT_FILE}")
    exit()

df_original = pd.read_excel(INPUT_FILE)
log(f"فایل اصلی خوانده شد → {len(df_original):,} ردیف")

# بررسی وجود ستون‌های مهم
required_cols = ['id', 'RecordDate', 'RecordTime', 'AssetID_9286']
missing = [col for col in required_cols if col not in df_original.columns]
if missing:
    log(f"هشدار: ستون‌های زیر پیدا نشدند: {missing}")
    log("اما ادامه می‌دیم...")

# ================================
# 2. آماده‌سازی داده برای مدل (فقط ستون‌های عددی)
# ================================
# ستون‌هایی که نباید تو مدل باشن
non_feature_cols = ['id', 'RecordDate', 'RecordTime']

# حذف موقت این ستون‌ها برای آموزش
model_df = df_original.drop(columns=[c for c in non_feature_cols if c in df_original.columns], errors='ignore')

target = 'AssetID_9286'
if target not in model_df.columns:
    log("خطا: ستون AssetID_9286 وجود ندارد!")
    exit()

X = model_df.drop(target, axis=1)
y = model_df[target]

log(f"تعداد فیچرهای استفاده شده در مدل: {X.shape[1]}")

# ================================
# 3. نرمالایز کردن
# ================================
log("نرمالایز کردن داده‌ها...")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

# ================================
# 4. تقسیم و آموزش مدل
# ================================
X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.2, random_state=42)

log("در حال آموزش XGBoost...")
model = XGBRegressor(
    n_estimators=1000,
    learning_rate=0.03,
    max_depth=7,
    subsample=0.9,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=1.0,
    random_state=42,
    n_jobs=-1,
    tree_method='hist'
)
model.fit(X_train, y_train)
log("آموزش XGBoost با موفقیت انجام شد.")

# ================================
# 5. ارزیابی مدل
# ================================
y_pred_test = model.predict(X_test)
mae  = mean_absolute_error(y_test, y_pred_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
r2   = r2_score(y_test, y_pred_test)
mape = np.mean(np.abs((y_test - y_pred_test) / (y_test + 1e-8))) * 100

log("\n" + "="*60)
log("               نتایج ارزیابی مدل")
log("="*60)
log(f"   MAE           : {mae:.8f}")
log(f"   RMSE          : {rmse:.8f}")
log(f"   R²            : {r2:.6f}")
log(f"   MAPE (%)      : {mape:.4f}%")
log("="*60)

# ================================
# 6. پیش‌بینی روی کل داده (با حفظ ترتیب اصلی)
# ================================
y_pred_full = model.predict(X_scaled_df)

# ================================
# 7. ساخت خروجی نهایی — دقیقاً با ترتیب اصلی + همه ستون‌های خواسته شده
# ================================
result = df_original.copy()  # شروع از فایل اصلی → ترتیب 100% حفظ میشه

# اضافه کردن نتایج پیش‌بینی
result['AssetID_9286_Predicted'] = y_pred_full
result['Error (Real - Predicted)'] = result['AssetID_9286'] - y_pred_full
result['Absolute_Error'] = np.abs(result['Error (Real - Predicted)'])
result['Percentage_Error (%)'] = np.abs(result['Error (Real - Predicted)'] / (result['AssetID_9286'] + 1e-8)) * 100

# ================================
# 8. ترتیب نهایی ستون‌ها — خیلی مرتب و حرفه‌ای
# ================================
# ستون‌های اول (مهم‌ترین‌ها)
priority_cols = []

# 1. RecordDate و RecordTime اول
if 'RecordDate' in result.columns:
    priority_cols.append('RecordDate')
if 'RecordTime' in result.columns:
    priority_cols.append('RecordTime')

# 2. id
if 'id' in result.columns:
    priority_cols.append('id')

# 3. مقدار واقعی و پیش‌بینی و خطاها
priority_cols += [
    'AssetID_9286',
    'AssetID_9286_Predicted',
    'Error (Real - Predicted)',
    'Absolute_Error',
    'Percentage_Error (%)'
]

# بقیه ستون‌ها (فیچرها)
remaining_cols = [col for col in result.columns if col not in priority_cols]
final_columns = priority_cols + remaining_cols

result = result[final_columns]

# ================================
# 9. ذخیره نهایی
# ================================
result.to_excel(OUTPUT_FILE, index=False, engine='openpyxl')

log(f"\nموفقیت‌آمیز! خروجی کامل با RecordDate و RecordTime ذخیره شد.")
log(f"فایل: {OUTPUT_FILE}")
log(f"ترتیب ردیف‌ها: 100% حفظ شده")
log(f"ستون‌های اول: RecordDate → RecordTime → id → AssetID_9286 → ...")

# نمایش نهایی
print("\n" + "="*80)
print("                 پیش‌بینی با موفقیت انجام شد!")
print("="*80)
print(f"   فایل خروجی: {OUTPUT_FILE}")
print(f"   تعداد ردیف: {len(result):,}")
print(f"   ستون‌های اول: {' → '.join(priority_cols[:6])} → ...")
print(f"   MAE          : {mae:.8f}")
print(f"   MAPE         : {mape:.4f}%")
print(f"   R²           : {r2:.6f}")
print("="*80)
print("   حالا فایل رو باز کن — همه چیز مرتب، کامل و حرفه‌ایه!")
print("="*80)

شروع پیش‌بینی AssetID_9286 — با RecordDate + RecordTime + id + حفظ ترتیب
فایل اصلی خوانده شد → 3,000 ردیف
تعداد فیچرهای استفاده شده در مدل: 6
نرمالایز کردن داده‌ها...
در حال آموزش XGBoost...
آموزش XGBoost با موفقیت انجام شد.

               نتایج ارزیابی مدل
   MAE           : 0.03769099
   RMSE          : 0.07890188
   R²            : 0.946204
   MAPE (%)      : 0.4955%

موفقیت‌آمیز! خروجی کامل با RecordDate و RecordTime ذخیره شد.
فایل: C:\BI\regression_output_for_assetid_9286.xlsx
ترتیب ردیف‌ها: 100% حفظ شده
ستون‌های اول: RecordDate → RecordTime → id → AssetID_9286 → ...

                 پیش‌بینی با موفقیت انجام شد!
   فایل خروجی: C:\BI\regression_output_for_assetid_9286.xlsx
   تعداد ردیف: 3,000
   ستون‌های اول: RecordDate → RecordTime → id → AssetID_9286 → AssetID_9286_Predicted → Error (Real - Predicted) → ...
   MAE          : 0.03769099
   MAPE         : 0.4955%
   R²           : 0.946204
   حالا فایل رو باز کن — همه چیز مرتب، کامل و حرفه‌ایه!
